# explore-1.ipynb

### CSc-59866 - Senior Design - Prof. Etemadpour

* Purpose: exploratory data analysis for shelter, COVID-19, and unemployment dataset from NYC Open Data and NYS Department of Labor
* Date: 2020-11-27
* Authors: Xin Chen, Ian S. McBride, Lifu Tao

### Data files:

Create a folder named `data` next to this notebook. Place the following CSV files in the `data` folder

* https://data.cityofnewyork.us/Social-Services/DHS-Daily-Report/k46n-sa2m - export as CSV
* https://data.cityofnewyork.us/Health/COVID-19-Daily-Counts-of-Cases-Hospitalizations-an/rc75-m7u3 - export as CSV
* https://www.labor.ny.gov/stats/LSLAUS.shtm - select the region "NYC" and download CSV file

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [ ]:
df_shelter = pd.read_csv('./data/DHS_Daily_Report.csv')
df_covid = pd.read_csv('./data/COVID-19_Daily_Counts_of_Cases__Hospitalizations__and_Deaths.csv')

In [ ]:
display('shelter', df_shelter.columns)
print()
display('covid', df_covid.columns)

In [ ]:
columns_orig = [
    'Date of Census',
    'Total Adults in Shelter',
    'Total Children in Shelter',
    'Total Individuals in Shelter',
    'Total Individuals in Families with Children in Shelter ',
    'Individuals in Adult Families in Shelter',
]
columns_renamed = [
    'Shelter Adults',
    'Shelter Children',
    'Shelter Total',
    'Shelter Individuals In Families With Children',
    'Shelter Individuals In Adult Families',
]

# Create df_shelter
df_shelter = pd.read_csv(
    './data/DHS_Daily_Report.csv',
    index_col='Date of Census',
    usecols=columns_orig,
)
display(df_shelter.columns)

df_shelter.columns = columns_renamed
df_shelter = df_shelter[~df_shelter.index.duplicated(keep='first')]
df_shelter.index = pd.to_datetime(df_shelter.index, format='%m/%d/%Y')
df_shelter.sort_index(inplace=True)
display(df_shelter)
display(df_shelter.info())

In [ ]:
# Create df_covid
columns_orig = [
    'DATE_OF_INTEREST',
    'CASE_COUNT',
    'HOSPITALIZED_COUNT',
    'DEATH_COUNT',
]
columns_renamed = [
    'COVID Cases',
    'COVID Hospitalizations',
    'COVID Deaths',
]
df_covid = pd.read_csv(
    'data/COVID-19_Daily_Counts_of_Cases__Hospitalizations__and_Deaths.csv',
    index_col='DATE_OF_INTEREST',
    usecols=columns_orig
)
df_covid.columns = columns_renamed
df_covid.index = pd.to_datetime(df_covid.index, format='%m/%d/%Y %I:%M:%S %p')
df_covid.sort_index(inplace=True)
display(df_covid)
display(df_covid.info())

In [ ]:
# Join shelter, covid dataframes
df_shelter_covid = pd.concat([df_shelter, df_covid], axis=1, sort=False)

# Drop dates in df_shelter don't overlap with df_covid
df_shelter_covid.dropna(inplace=True)

display(df_shelter_covid.info())

In [ ]:
filename_labor = './data/lausCSV.csv'
with open(filename_labor, mode='r') as f:
    lines = f.readlines()

# Grab rows with dataset title, column names, and two years
select_file_rows = lambda lines: [lines[idx-2:idx+2] for idx, line in enumerate(lines) if line.startswith('2020')]

# Split row strings into lists, reverse the order (old to new), remove three columns (year, annual average, and trailing empty column)
split_rows_into_list_by_year = lambda rows: {
    variable[0].strip().title(): {l[0]: l[1:-2] for l in list(map(lambda x: x.strip().split(','), variable[-1:1:-1]))}
    for variable in rows
}

# Concat years into one list per variable all in one dict
concat_years_into_one_list = lambda variables: {
    variable_name: [value for lst in d.values() for value in lst]
    for variable_name, d in variables.items()
}

# Create dataframe from dict of variables
def create_df(variables):
    df = pd.DataFrame(
        variables,
        index=pd.date_range(start='2019-01-01', end='2020-12-31', freq='m')
    )
    # Remove NaN values from dataframe
    df = df.replace('', np.nan).dropna()
    
    # Fix one column's datatype
    df['Unemployment Rate'] = df['Unemployment Rate'].transform(
        lambda x: float(x.strip('%')) / 100 if type(x) is str and x else np.nan,
        axis=0
    )
    
    # Fix type of remaining columns
    types = {
        'Unemployment Rate': float,
        'Labor Force': int,
        'Employed': int,
        'Unemployed': int,
    }
    df = df.astype(types)
    
    return df

df_labor = create_df(
    concat_years_into_one_list(
        split_rows_into_list_by_year(
            select_file_rows(
                lines))))

# Note that all columns are object type
display(df_labor.info())
display(df_labor)

In [ ]:
fig = plt.figure(figsize=(16, 16))

# Shelter
ax = fig.add_subplot(2, 2, 1)
_ = df_shelter_covid.plot(
    ax=ax,
    y=[
        'Shelter Adults',
        'Shelter Children',
        'Shelter Total',
    ]
)

# COVID
ax = fig.add_subplot(2, 2, 2)
_ = df_shelter_covid.plot(
    ax=ax,
    y=[
        'COVID Cases',
        'COVID Hospitalizations',
        'COVID Deaths',
    ]
)

# Labor
ax = fig.add_subplot(2, 2, 3)
_ = df_labor.loc['2020-02-29':].plot(
    ax=ax,
    y=[
        'Unemployment Rate',
    ]
)